In [1]:
from cell_env import CellEnv
# Use sb3 env checker:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import DQN

from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback


In [2]:
env = CellEnv()
check_env(env)
# use the monitor wrapper to log the results:
env = Monitor(env)
eval_env = CellEnv(dt=0.1)
eval_env = Monitor(eval_env)

In [3]:

eval_callback = EvalCallback(eval_env, best_model_save_path='./rl-models-sde/',
                             n_eval_episodes=25,
                             log_path='./rl-logs/', eval_freq=10_000,
                             deterministic=True, render=False,
                             )

model = DQN("MlpPolicy", DummyVecEnv([lambda: Monitor(env)]), verbose=4, tensorboard_log="./rl-logs/",
            exploration_fraction=0.2,
            target_update_interval=2000,
            buffer_size=10_000,
            gradient_steps=2,
            learning_starts=1000,
            learning_rate=0.0015,
            batch_size=16,
)
model.learn(total_timesteps=100_000, tb_log_name="dqn",
            callback=eval_callback)


Using cuda device
Logging to ./rl-logs/dqn_8
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 100      |
|    ep_rew_mean      | -129     |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1268     |
|    time_elapsed     | 0        |
|    total_timesteps  | 400      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 100      |
|    ep_rew_mean      | -68.1    |
|    exploration_rate | 0.987    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1268     |
|    time_elapsed     | 0        |
|    total_timesteps  | 800      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 100      |
|    ep_rew_mean      | -49.7    |
|    exploration_rate | 0.981    |
| time/   